In [1]:
# Python Script to compare libraries of Antibody DNA sequences to PG9 VDJ wild type
# Convert to amino acid sequence to look to enrichment of mutations in libraries (1 - 4) 
# compared to control library

In [2]:
# Etract PG9 DNA sequence (a string made up of the geneteic code consisting of A, T, C, G)

with open("source_sequence.fasta.txt", "r") as source:
    for line in source:
        if not line.startswith('>'):
            PG9_seq = line.strip()
            
print(PG9_seq)

CAGCGATTAGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGTCGTCCCTGAGACTCTCCTGTGCAGCGTCCGGATTCGACTTCAGTAGACAAGGCATGCACTGGGTCCGCCAGGCTCCAGGCCAGGGGCTGGAGTGGGTGGCATTTATTAAATATGATGGAAGTGAGAAATATCATGCTGACTCCGTATGGGGCCGACTCAGCATCTCCAGAGACAATTCCAAGGATACGCTTTATCTCCAAATGAATAGCCTGAGAGTCGAGGACACGGCTACATATTTTTGTGTGAGAGAGGCTGGTGGGCCCGACTACCGTAATGGGTACAACTATTACGATTTCTATGATGGTTATTATAACTACCACTATATGGACGTCTGGGGCAAAGGGACCACGGTCACCGTCTCGAGC


In [4]:
# extract DNA sequences from control or mutated libraries - FASTA files (each DNA sequence assigned a fasta name in the library)

# create a dictionary with fasta names as keys and DNA sequences as values

keys = []
values = []

# loop through library and extract fasta names as keys for dictionary and then the DNA sequences as the values

with open("control_library.fasta.txt", "r") as lib:
    for line in lib:
        if line.startswith('>'):
            keys.append(line.strip().lstrip(">"))
        else:
            values.append(line.strip())

DNA_lib = dict(zip(keys, values)) # library in dictionary format with fasta names as keys and sequences as values

print ("Example Sequence:", DNA_lib['74a74b4e-e0c6-4fb7-bf10-cf36fa80638c'])
print ("Total number of sequences in file",len(keys)) #total number of DNA sequences (strings) in the file

Example Sequence: CAGCGATTAGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGTCGTCCCTGAGACTCTCCTGTGCAGCGTCCGGATTCGACTTCAGTAGACAAGGCATGCACTGGGTCCGCCAGGCTCCAGGCCAGGGGCTGGAGTGGGTGGCATTTATTAAATATGATGGAAGTGAGAAATATCATGCTGACTCCGTATGGGGCCGACTCAGCATCTCCAGAGACAATTCCAAGGATACGCTTTATCTCCAAATGAATAGCCTGAGAGTCGAGGACACGGCTACATATGTATGTGTGAGAGAGGCTGGTGGGCCCGACTACCGTAATGGGTACAACTATTACGATTTCTATGATGGTTATTATAACTACCACTATATGGACGTCTGGGGCAAAGGGACCACGGTCACCGTCTCGAGC
Total number of sequences in file 100000


In [7]:
# create codon:amino acid dictionary (codons (triplet DNA code) as keys and single amino acid letter as the value in the dictionary)
# single amino acid code (a letter) is obtained from the triplet DNA code - converting one string to another

bases = ['T', 'C', 'A', 'G']
codons = [a+b+c for a in bases for b in bases for c in bases] #creat list of DNA codons
amino_acids = 'FFLLSSSSYY**CC*WLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG'  #STOP = *
codon_table = dict(zip(codons, amino_acids))

# How codon loop works:
# first loop - a and b remain constant (T) and c changes: TTT, TTC, TTA, TTG
# second loop - a is T, b is C and c changes: TCT TCC TCA TCG

print(codon_table)

{'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L', 'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S', 'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*', 'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W', 'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L', 'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P', 'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M', 'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T', 'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R', 'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V', 'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A', 'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}


In [8]:
# function to convert codon to single letter amino acid code

def translate(codon):
    for key, value in codon_table.items():
        if codon == key:
            return value
    
print(translate('ATG')) #test function

M


In [15]:
# loop through dictionary containing many DNA sequences (100000 in a library)
# compare DNA sequences to PG9 sequence and output mutated codons, translate to amino acids and get positions of the mutations
# compare DNA sequences in library (strings) to PG9 sequence (also a string) as well as convert to amino acid sequence (another string)

key_fasta = []  #store keys - fasta names
aa = []   #store amino acid (translate codon after moving through sequence every 3 nucleotides and compare to PG9)
pos_aa = []  #store amino acid position


for key, value in DNA_lib.items():  #loop through library of DNA sequences
    for pos in range(0, len(value)+1, 3):  # move through sequence (values) every 3 nucleotides (codons)
        if value[pos:pos+3] != PG9_seq[pos:pos+3]:
            key_fasta.append(key)
            aa.append(translate(value[pos:pos+3]))
            pos_aa.append((pos+3)/3)
            
aa_mut = dict(zip(key_fasta, aa))  #keys (fasta file name) with corresponding mutated amino acid
pos_mut = dict(zip(key_fasta, pos_aa))#same keys as above with corresponding amino acid position

In [17]:
# Count the number of times each mutated postion occurs - 
# output is (position, number of sequences with a mutation at that position)

from collections import Counter

unique = Counter(pos_aa)
print ("Amino acid position with number of sequences mutated at that position:")
print (unique.items())

print(" ")
print ("Total number of sequences in library:", len(keys))
print ("Total number of sequences with mutations:", len(key_fasta))
print ("Number of unmutated sequences:", len(keys)-len(key_fasta))

Amino acid position with number of sequences mutated at that position:
dict_items([(94.0, 9833), (28.0, 9809), (112.0, 9732), (79.0, 9751), (76.0, 9841), (10.0, 9742), (13.0, 9753), (128.0, 9950), (55.0, 9890), (78.0, 10034)])
 
Total number of sequences in library: 100000
Total number of sequences with mutations: 98335
Number of unmutated sequences: 1665


In [20]:
# function to output a list of positions mutated - unique

def pos_mutated(pos_aa):
    unique = []
    for x in pos_aa:
        if x not in unique:
            unique.append(x)
    return unique


track = pos_mutated(pos_aa) #track the positions mutated
print ("Amino acid positions at which mutations occur:",track)
print ("Total number of positions mutated in every sequence:",len(track))

Amino acid positions at which mutations occur: [94.0, 28.0, 112.0, 79.0, 76.0, 10.0, 13.0, 128.0, 55.0, 78.0]
Total number of positions mutated in every sequence: 10


In [21]:
# generate multiple empty lists using the position number as the key - 
# to append keys (fasta name) sorted by position of mutation

pos_keys_lists = {}

for i in range(len(track)):
    pos_keys_lists[track[i]] = []
    
    
print (pos_keys_lists)

{94.0: [], 28.0: [], 112.0: [], 79.0: [], 76.0: [], 10.0: [], 13.0: [], 128.0: [], 55.0: [], 78.0: []}


In [25]:
# extract keys/fasta file names for each mutated position and store in pos_keys_lists dictionary
# store all keys/fasta file names corresponding to a particular position where position is the key 
# and keys (values) are stored as a list

for x in track:
    for key, value in pos_mut.items(): #dictionary of fasta file names (keys) with associated amino acid positions
        if pos_mut[key] == x:              #if value (amino acid position) = position from track (x)
            pos_keys_lists[x].append(key)  #save fasta file name (key) in list with position as key 

#test code
print ("Number of sequences with mutations at position 78:",len(pos_keys_lists[78]))
print ("Number of sequences with mutations at position 128:",len(pos_keys_lists[128]))
print ("Number of sequences with mutations at position 94:",len(pos_keys_lists[94]))

Number of sequences with mutations at position 78: 20068
Number of sequences with mutations at position 128: 19900
Number of sequences with mutations at position 94: 19666


In [26]:
# generate multiple empty lists using position number as key - 
# store mutated ammino acids for that position

aa_lists = {}

for i in range(len(track)):
    aa_lists[track[i]] = []
    
    
print (aa_lists)

{94.0: [], 28.0: [], 112.0: [], 79.0: [], 76.0: [], 10.0: [], 13.0: [], 128.0: [], 55.0: [], 78.0: []}


In [28]:
# using keys extracted according to position, now grab all the mutated amino acids for that position

for x in track:
    for key, value in aa_mut.items(): #dict of fasta file names/keys with corresponding amino acid
        if key in pos_keys_lists[x]:        #if fasta file name (key) in list corresponding to position key (x)
            aa_lists[x].append(aa_mut[key]) #grab the values (mutated amino acid) into list corresponding to position key (x)
            
            
# use Counter to get number of each amino acid for positions
for x in track:
    aa_pos = Counter(aa_lists[x])
    print ("Position:", x)
    print ("Total number of sequences for position", x, ":", len(aa_lists[x]))
    print ("Number of times a certain amino acid occurs at this position:")
    print (aa_pos.items())
    print(" ")

Position: 94.0
Total number of sequences for position 94.0 : 19666
Number of times a certain amino acid occurs at this position:
dict_items([('V', 1342), ('M', 362), ('H', 702), ('I', 982), ('L', 2010), ('S', 1904), ('G', 1238), ('N', 666), ('R', 1938), ('Y', 634), ('T', 1306), ('C', 642), ('A', 1376), ('E', 750), ('Q', 578), ('P', 1320), ('K', 654), ('D', 628), ('W', 300), ('F', 334)])
 
Position: 28.0
Total number of sequences for position 28.0 : 19618
Number of times a certain amino acid occurs at this position:
dict_items([('R', 1852), ('E', 702), ('T', 1272), ('Y', 640), ('W', 326), ('C', 594), ('Q', 664), ('L', 1946), ('I', 996), ('H', 650), ('A', 1358), ('F', 306), ('G', 1422), ('P', 1356), ('V', 1238), ('N', 696), ('S', 1934), ('D', 706), ('K', 652), ('M', 308)])
 
Position: 112.0
Total number of sequences for position 112.0 : 19464
Number of times a certain amino acid occurs at this position:
dict_items([('L', 2008), ('Y', 686), ('N', 628), ('A', 1346), ('C', 728), ('V', 1292)

In [29]:
# function to count amino acids in a list

a_list = ['F', 'L', 'S', 'Y', 'C', 'W', 'P', 'H', 'Q', 'R', 'I', 'M', 'T', 'N', 'K', 'V', 'A', 'D', 'E', 'G']


def count_aa(aa):
    aa_count = {} # generate dictionary with amnio acids as key to keep count of each aa
    for i in range(len(a_list)):
        aa_count[a_list[i]] = []
        
    for x in a_list:
        count = 0
        for y in aa:
            if y == x:
                count += 1
        aa_count[x].append(count)
    return aa_count
        
aa_test = ['F', 'F', 'A', 'H', 'H', 'H']
print (count_aa(aa_test))

{'F': [2], 'L': [0], 'S': [0], 'Y': [0], 'C': [0], 'W': [0], 'P': [0], 'H': [3], 'Q': [0], 'R': [0], 'I': [0], 'M': [0], 'T': [0], 'N': [0], 'K': [0], 'V': [0], 'A': [1], 'D': [0], 'E': [0], 'G': [0]}


In [31]:
# count the number of amino acids at each position

aa_count_lib = {} # generate dictionary with positions as key to store lists of amino acid total counts for each position

for i in range(len(track)):
    aa_count_lib[track[i]] = []


for x in track:                                #track positions - keys in aa_count_lib
    for key, value in aa_lists.items():    #dict with positions as keys and list of mutated amino acids at that position
        if key == x:
            aa_count_lib[x].append(count_aa(value)) #count amino acids at position x
            
print ("Amino acid counts at each position:" )           
for x in track:
    print ("Position", x)
    print (aa_count_lib[x])
    print ("Total number of sequences for position", x ,":", len(aa_lists[x]))
    print(" ")

Amino acid counts at each position:
Position 94.0
[{'F': [334], 'L': [2010], 'S': [1904], 'Y': [634], 'C': [642], 'W': [300], 'P': [1320], 'H': [702], 'Q': [578], 'R': [1938], 'I': [982], 'M': [362], 'T': [1306], 'N': [666], 'K': [654], 'V': [1342], 'A': [1376], 'D': [628], 'E': [750], 'G': [1238]}]
Total number of sequences for position 94.0 : 19666
 
Position 28.0
[{'F': [306], 'L': [1946], 'S': [1934], 'Y': [640], 'C': [594], 'W': [326], 'P': [1356], 'H': [650], 'Q': [664], 'R': [1852], 'I': [996], 'M': [308], 'T': [1272], 'N': [696], 'K': [652], 'V': [1238], 'A': [1358], 'D': [706], 'E': [702], 'G': [1422]}]
Total number of sequences for position 28.0 : 19618
 
Position 112.0
[{'F': [644], 'L': [2008], 'S': [1902], 'Y': [686], 'C': [728], 'W': [334], 'P': [1210], 'H': [632], 'Q': [578], 'R': [1916], 'I': [946], 'M': [392], 'T': [1358], 'N': [628], 'K': [646], 'V': [1292], 'A': [1346], 'D': [308], 'E': [648], 'G': [1262]}]
Total number of sequences for position 112.0 : 19464
 
Posit

In [32]:
# function to get amino acid frequency

def freq_aa(aa):
    aa_freq = {} # generate dictionary with amnio acids as key to keep count of each aa
    for i in range(len(a_list)):
        aa_freq[a_list[i]] = []
        
    for x in a_list:
        count = 0.0
        for y in aa:
            total = len(aa)  #length of the list, that is, total number of mutated amino acids at that position
            if y == x:
                count += 1.0
        aa_freq[x].append("%.4f"%(count/total))
    return aa_freq
        
aa_test = ['F', 'F', 'A', 'H', 'H', 'H']
print (freq_aa(aa_test))

{'F': ['0.3333'], 'L': ['0.0000'], 'S': ['0.0000'], 'Y': ['0.0000'], 'C': ['0.0000'], 'W': ['0.0000'], 'P': ['0.0000'], 'H': ['0.5000'], 'Q': ['0.0000'], 'R': ['0.0000'], 'I': ['0.0000'], 'M': ['0.0000'], 'T': ['0.0000'], 'N': ['0.0000'], 'K': ['0.0000'], 'V': ['0.0000'], 'A': ['0.1667'], 'D': ['0.0000'], 'E': ['0.0000'], 'G': ['0.0000']}


In [33]:
# get the frequency of amino acids at each position for control library - baseline frequencies

aa_freq_c = {} # generate dictionary with positions as key to store lists of amino acid total counts for each position

for i in range(len(track)):
    aa_freq_c[track[i]] = []


for x in track:
    for key, value in aa_lists.items():
        if key == x:
            aa_freq_c[x].append(freq_aa(value))
            

print ("Amino acid frequencies at each position:")            
for x in track:
    print ("Position", x)
    print (aa_freq_c[x])
    print(" ")

Amino acid frequencies at each position:
Position 94.0
[{'F': ['0.0170'], 'L': ['0.1022'], 'S': ['0.0968'], 'Y': ['0.0322'], 'C': ['0.0326'], 'W': ['0.0153'], 'P': ['0.0671'], 'H': ['0.0357'], 'Q': ['0.0294'], 'R': ['0.0985'], 'I': ['0.0499'], 'M': ['0.0184'], 'T': ['0.0664'], 'N': ['0.0339'], 'K': ['0.0333'], 'V': ['0.0682'], 'A': ['0.0700'], 'D': ['0.0319'], 'E': ['0.0381'], 'G': ['0.0630']}]
 
Position 28.0
[{'F': ['0.0156'], 'L': ['0.0992'], 'S': ['0.0986'], 'Y': ['0.0326'], 'C': ['0.0303'], 'W': ['0.0166'], 'P': ['0.0691'], 'H': ['0.0331'], 'Q': ['0.0338'], 'R': ['0.0944'], 'I': ['0.0508'], 'M': ['0.0157'], 'T': ['0.0648'], 'N': ['0.0355'], 'K': ['0.0332'], 'V': ['0.0631'], 'A': ['0.0692'], 'D': ['0.0360'], 'E': ['0.0358'], 'G': ['0.0725']}]
 
Position 112.0
[{'F': ['0.0331'], 'L': ['0.1032'], 'S': ['0.0977'], 'Y': ['0.0352'], 'C': ['0.0374'], 'W': ['0.0172'], 'P': ['0.0622'], 'H': ['0.0325'], 'Q': ['0.0297'], 'R': ['0.0984'], 'I': ['0.0486'], 'M': ['0.0201'], 'T': ['0.0698'], 'N'

In [36]:
# function to get amino acid frequency as a percentage

def freq_aa100(aa):
    aa_freq100 = {} # generate dictionary with amnio acids as key to keep count of each aa
    for i in range(len(a_list)):
        aa_freq100[a_list[i]] = []
        
    for x in a_list:
        count = 0.0
        freq100 = 0.0
        for y in aa:
            total = len(aa)  #length of the list, that is, total number of mutated amino acids at that position
            if y == x:
                count += 1.0
                freq100 = "%.4f%%" % (100 * (count/total))
        aa_freq100[x].append(freq100)
    return aa_freq100
        
aa_test = ['F', 'F', 'A', 'H', 'H', 'H']
print (freq_aa100(aa_test))

{'F': ['33.3333%'], 'L': [0.0], 'S': [0.0], 'Y': [0.0], 'C': [0.0], 'W': [0.0], 'P': [0.0], 'H': ['50.0000%'], 'Q': [0.0], 'R': [0.0], 'I': [0.0], 'M': [0.0], 'T': [0.0], 'N': [0.0], 'K': [0.0], 'V': [0.0], 'A': ['16.6667%'], 'D': [0.0], 'E': [0.0], 'G': [0.0]}


In [35]:
# get the frequency of amino acids at each position for control library - baseline frequencies, as a percentage

aa_freq_c100 = {} # generate dictionary with positions as key to store lists of amino acid total counts for each position

for i in range(len(track)):
    aa_freq_c100[track[i]] = []


for x in track:
    for key, value in aa_lists.items():
        if key == x:
            aa_freq_c100[x].append(freq_aa100(value))
            

print ("Amino acid frequencies at each position:")            
for x in track:
    print ("Position", x)
    print (aa_freq_c100[x])
    print(" ")

Amino acid frequencies at each position:
Position 94.0
[{'F': ['1.6984%'], 'L': ['10.2207%'], 'S': ['9.6817%'], 'Y': ['3.2238%'], 'C': ['3.2645%'], 'W': ['1.5255%'], 'P': ['6.7121%'], 'H': ['3.5696%'], 'Q': ['2.9391%'], 'R': ['9.8546%'], 'I': ['4.9934%'], 'M': ['1.8407%'], 'T': ['6.6409%'], 'N': ['3.3866%'], 'K': ['3.3255%'], 'V': ['6.8240%'], 'A': ['6.9968%'], 'D': ['3.1933%'], 'E': ['3.8137%'], 'G': ['6.2951%']}]
 
Position 28.0
[{'F': ['1.5598%'], 'L': ['9.9195%'], 'S': ['9.8583%'], 'Y': ['3.2623%'], 'C': ['3.0278%'], 'W': ['1.6617%'], 'P': ['6.9120%'], 'H': ['3.3133%'], 'Q': ['3.3846%'], 'R': ['9.4403%'], 'I': ['5.0770%'], 'M': ['1.5700%'], 'T': ['6.4838%'], 'N': ['3.5478%'], 'K': ['3.3235%'], 'V': ['6.3105%'], 'A': ['6.9222%'], 'D': ['3.5987%'], 'E': ['3.5783%'], 'G': ['7.2484%']}]
 
Position 112.0
[{'F': ['3.3087%'], 'L': ['10.3165%'], 'S': ['9.7719%'], 'Y': ['3.5245%'], 'C': ['3.7402%'], 'W': ['1.7160%'], 'P': ['6.2166%'], 'H': ['3.2470%'], 'Q': ['2.9696%'], 'R': ['9.8438%'], 'I